# Link Prediction in Social Network Analysis (SNA)

### Introduction
In this notebook, we will explore **Link Prediction** in Social Network Analysis using machine learning techniques. Social networks are composed of individuals (nodes) connected by relationships (edges), and link prediction aims to predict future or missing connections based on the current structure of the network.

Using **Zachary's Karate Club** dataset (a classic social network graph), we will:
- Visualize the original social network,
- Extract features from the network (such as common neighbors),
- Train a machine learning model to predict missing links,
- Evaluate the model's accuracy, and
- Visualize the predicted links in the network.

The notebook will demonstrate how to implement basic link prediction using the **Random Forest** classifier from `scikit-learn`, along with the **NetworkX** library for network visualization and manipulation.

By the end of this notebook, you'll understand how link prediction can be applied in social networks and how machine learning helps uncover hidden or future relationships between individuals.

### Key Steps:
1. **Graph Creation and Visualization**: We'll start by visualizing the network structure of Zachary's Karate Club.
2. **Data Preparation**: We'll split the network into existing links (edges) and missing links (non-edges), which will serve as the dataset for training our model.
3. **Feature Extraction**: We'll calculate features like common neighbors between pairs of nodes to help the model predict new links.
4. **Model Training**: A Random Forest classifier will be trained on the existing data to learn the patterns of link formation.
5. **Prediction and Evaluation**: We'll test the model on unseen data to predict which non-existent links are likely to form, and evaluate its performance.
6. **Visualization of Predictions**: Finally, we'll visualize the predicted links as red dashed lines in the graph to show potential new connections.

Let’s get started!


In [ ]:
# Link Prediction in Social Network Analysis using Jupyter Notebook

# Step 1: Install Necessary Libraries

#!pip install networkx scikit-learn matplotlib



### Step 2: Import Required Libraries

In this step, we will import the necessary Python libraries that will enable us to work with graphs, perform machine learning tasks, and visualize the results.

#### Libraries used:
1. **NetworkX**: This is a popular library for the creation, manipulation, and study of complex networks or graphs. We will use it to load and manipulate the social network (Zachary's Karate Club) and extract graph-based features for our link prediction task.
   
2. **Matplotlib**: A widely used library for creating static, interactive, and animated visualizations in Python. We will use it to visualize the graph and the predicted links.

3. **Scikit-learn**: This is a versatile machine learning library in Python. We'll use it to:
   - Split the data into training and testing sets,
   - Train a Random Forest classifier for predicting links, and
   - Evaluate the model's performance.

By importing these libraries, we ensure that we have the tools required for working with social network data, building machine learning models, and generating visualizations.



In [ ]:
# Step 2: Import Required Libraries
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



### Step 3: Create a Graph with NetworkX

In this step, we will create and visualize the social network using the **NetworkX** library.

#### Graph: Zachary's Karate Club
We are using **Zachary's Karate Club Graph**, a well-known social network dataset in network analysis. This graph represents friendships between 34 members of a karate club, where nodes represent individuals, and edges represent friendships between them.

#### Key Tasks:
1. **Loading the Graph**: We use the `nx.karate_club_graph()` function from NetworkX to load this pre-built graph dataset. It's a great example for link prediction tasks due to its small size and real-world structure.
   
2. **Positioning Nodes**: To visualize the graph, we need to compute positions for each node. We use the `spring_layout()` function, which positions the nodes based on a force-directed algorithm, creating an intuitive and easy-to-understand graph layout.

3. **Graph Visualization**: We will visualize the network using `Matplotlib`. The graph will display:
   - **Nodes**: Each node represents an individual in the club.
   - **Edges**: Edges between nodes represent existing friendships.

Visualizing the graph gives us a clear picture of the network structure, allowing us to understand the relationships between individuals before we proceed to predict new or missing links.

#### Why Visualization is Important:
By visualizing the graph, we can observe the structure of relationships within the network and detect any patterns or clusters. This step helps in understanding the data before moving to machine learning tasks.

Let’s go ahead and visualize Zachary’s Karate Club!



In [ ]:
# Step 3: Create a Graph with NetworkX (Zachary's Karate Club)
G = nx.karate_club_graph()

# Generate positions for the nodes using spring layout
pos = nx.spring_layout(G)

# Visualize the graph (optional)
plt.figure(figsize=(8,6))
nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=500, font_size=10)
plt.title("Zachary's Karate Club Graph")
plt.show()




### Step 4: Split Data for Link Prediction

In this step, we will prepare our dataset for the link prediction task by splitting it into existing edges (links) and non-edges (missing links).

#### Key Tasks:
1. **Extract Existing Edges**: We retrieve all existing edges from the graph using the `G.edges()` function. These edges represent the actual friendships in Zachary's Karate Club.

2. **Identify Non-Edges**: Using `nx.non_edges(G)`, we generate a list of potential connections that do not currently exist in the graph. These non-edges serve as our candidates for prediction.

3. **Split Data**: We divide both the existing edges and non-edges into training and testing sets using the `train_test_split` function from `scikit-learn`. This ensures that our model can learn from one set of data and be evaluated on another, helping us assess its predictive capabilities accurately.

By separating our dataset into training and testing sets, we prepare for the feature extraction and model training steps that follow, laying the groundwork for our link prediction model.

---

In [ ]:
# Step 4: Split Data for Link Prediction
# Get all existing edges (links) in the graph
edges = list(G.edges())
non_edges = list(nx.non_edges(G))

# Split the edges and non-edges into training and test sets
edges_train, edges_test = train_test_split(edges, test_size=0.3, random_state=42)
non_edges_train, non_edges_test = train_test_split(non_edges, test_size=0.3, random_state=42)


### Step 5: Feature Extraction (Common Neighbors)

In this step, we will extract features from the graph that will be used for training our machine learning model.

#### Key Tasks:
1. **Common Neighbors**: One of the simplest yet effective features for link prediction is the number of **common neighbors** between two nodes. The more common friends two individuals have, the more likely they are to form a connection. We will define a function, `common_neighbors(u, v, G)`, that counts the common neighbors between nodes `u` and `v` in graph `G`.

2. **Creating the Feature Matrix**: We will create a feature matrix using the `create_features` function. For each pair of nodes (both existing edges and non-edges), this function will calculate the number of common neighbors and label them as `1` for existing edges and `0` for non-edges.

This feature extraction step is crucial because it transforms the graph's structural information into a format suitable for machine learning. The features will help our model learn patterns that indicate whether a link is likely to exist between two nodes.

---

In [ ]:
# Step 5: Feature Extraction (Common Neighbors)
# Function to compute the number of common neighbors between two nodes
def common_neighbors(u, v, G):
    return len(list(nx.common_neighbors(G, u, v)))

# Create feature matrix for training (using common neighbors)
def create_features(edges, label, G):
    features = []
    for u, v in edges:
        features.append([common_neighbors(u, v, G), label])
    return features

# Extract features for training data (1 for existing edges, 0 for non-existing edges)
features_pos_train = create_features(edges_train, 1, G)
features_neg_train = create_features(non_edges_train, 0, G)

# Combine positive and negative examples
train_data = features_pos_train + features_neg_train

### Step 6: Train a Machine Learning Model

In this step, we will train a machine learning model using the features extracted in the previous step.

#### Key Tasks:
1. **Prepare the Dataset**: We separate our features and labels into `X_train` and `y_train`. `X_train` will contain the feature values (number of common neighbors), and `y_train` will consist of the labels (1 for existing edges, 0 for non-edges).

2. **Random Forest Classifier**: We will use a **Random Forest Classifier** from the `scikit-learn` library. Random Forest is a versatile and robust ensemble learning method that combines multiple decision trees to improve prediction accuracy.

3. **Model Training**: The classifier will be trained using the `fit()` method on our training dataset (`X_train`, `y_train`). This process allows the model to learn the relationship between the features and the presence or absence of edges in the graph.

By training our model, we prepare it to make predictions about which non-edges are likely to become edges based on the learned patterns from the training data.

---

In [ ]:
# Step 6: Train a Machine Learning Model
# Prepare the dataset
X_train = [x[:-1] for x in train_data]  # Features (common neighbors)
y_train = [x[-1] for x in train_data]   # Labels (1 for edge, 0 for no edge)

# Train a RandomForest classifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

### Step 7: Predict and Evaluate

In this step, we will use the trained model to make predictions and evaluate its performance.

#### Key Tasks:
1. **Extract Features for Testing**: We will extract features for both the test data (existing edges) and the non-edges using the `create_features` function. This will provide us with the data needed for evaluation.

2. **Prepare Test Dataset**: Similar to the training step, we will create `X_test` and `y_test` from the test data. `X_test` will hold the features, and `y_test` will contain the actual labels.

3. **Make Predictions**: We will use the `predict()` method of the trained classifier to predict the presence of edges in the test set.

4. **Evaluate Performance**: Finally, we will assess the model's performance using accuracy metrics with the `accuracy_score` function. This will give us a clear indication of how well the model performs in predicting links.

By evaluating the model's predictions against the true labels, we gain insight into its effectiveness and the reliability of our link prediction approach.

---

In [ ]:
# Step 7: Predict and Evaluate
# Extract features from test data
features_pos_test = create_features(edges_test, 1, G)
features_neg_test = create_features(non_edges_test, 0, G)
test_data = features_pos_test + features_neg_test

# Prepare test dataset
X_test = [x[:-1] for x in test_data]
y_test = [x[-1] for x in test_data]

# Predict on the test data
y_pred = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of Link Prediction: {accuracy:.2f}")

### Step 8: Visualize Some of the Predicted Links (Optional)

In this final step, we will visualize the results of our link prediction task by displaying the predicted links on the original graph.

#### Key Tasks:
1. **Identify Predicted Links**: We will filter the predicted results to find the non-edges that were classified as likely to form links (predicted edges).

2. **Visualize the Original Graph**: Using `Matplotlib`, we will visualize the original graph structure again.

3. **Overlay Predicted Links**: The predicted links will be displayed as **red dashed lines** on the graph. This visualization highlights potential new connections that the model suggests could exist based on the patterns learned from the training data.

4. **Interpretation**: By examining the predicted links in the context of the original graph, we can gain insights into potential future relationships within the network.

Visualizing the predictions helps us understand the model's output better and provides a clear view of how link prediction can inform us about potential connections in social networks.

In [ ]:
# Step 8: Visualize some of the predicted links (optional)
# Show true links vs predicted non-links
predicted_edges = [e for e, p in zip(non_edges_test, y_pred) if p == 1]

# Visualize the original graph with predicted links
plt.figure(figsize=(8,6))
nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=500, font_size=10)
nx.draw_networkx_edges(G, pos, edgelist=predicted_edges, edge_color='red', style='dashed', width=2)
plt.title("Original Graph with Predicted Links (in red dashed lines)")
plt.show()

Predicted links (red) that were not part of the original graph but were predicted by the machine learning model (in this case, a Random Forest classifier).

### Explanation of Results

In our link prediction task, we trained a model to guess which connections might form between people in Zachary's Karate Club. Here’s what we found:

1. **Predicted Links**: The model predicted some friendships that do not currently exist. These are potential connections that could happen in the future based on existing relationships.

2. **Performance**: We evaluated how well our model did by comparing its predictions to the actual friendships. 
   - **Accuracy**: The model's accuracy score tells us how many of the predicted friendships were correct. A higher score means the model is better at predicting new connections.

3. **Visual Results**: In the graph, we can see the original friendships represented by solid lines. The predicted friendships are shown as red dashed lines. 
   - **What it means**: The red dashed lines highlight possible new friendships that the model thinks could form based on shared friends.

In simple terms, our model helps us understand who might become friends in the future based on who they already know. The predictions can help us see patterns in social networks and make informed guesses about future relationships.
